# Pinecone Vector Store - Metadata Filter

In [ ]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

Build a Pinecone Index and connect to it

In [ ]:
import pinecone

api_key = os.environ['PINECONE_API_KEY']
pinecone.init(api_key=api_key, environment="eu-west1-gcp")

In [ ]:
# dimensions are for text-embedding-ada-002
pinecone.create_index("quickstart-index", dimension=1536, metric="euclidean", pod_type="p1")

In [ ]:
pinecone_index = pinecone.Index("quickstart-index")

Build the PineconeVectorStore and GPTVectorStoreIndex

In [ ]:
from llama_index import GPTVectorStoreIndex, StorageContext
from llama_index.vector_stores import PineconeVectorStore

In [ ]:
from llama_index.data_structs.node import Node

nodes = [
    Node('The Shawshank Redemption', extra_info={
        "author": "Stephen King",
        "theme": "Friendship",
    }),
    Node('The Godfather', extra_info={
        "director": "Francis Ford Coppola",
        "theme": "Mafia",
    }),
    Node("Inception", extra_info={
        "director": "Christopher Nolan",
    })
]

In [ ]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index, namespace='test_05_14')
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex(nodes, storage_context=storage_context)

Define metadata filters

In [ ]:
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters
filters = MetadataFilters(filters=[ExactMatchFilter(key='theme', value='Mafia')])

Retrieve from vector store with filters

In [ ]:
retriever = index.as_retriever(filters=filters)
retriever.retrieve('What is inception about?')

Use keyword arguments specific to pinecone

In [ ]:
retriever = index.as_retriever(vector_store_kwargs={"filter": {"theme": "Mafia"}})
retriever.retrieve('What is inception about?')